In [ ]:
import pandas as pd
df = pd.read_csv('/content/instagram dataset_comments.csv')
df.head()

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_text(text):

    if not isinstance(text, str):
        text = str(text)
    return tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='tf')

df['tokenized_comments'] = df['comments'].apply(tokenize_text)
df['tokenized_statements'] = df['statements'].apply(tokenize_text)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Bidirectional, GRU, Dense, Concatenate
from transformers import TFBertModel, BertTokenizer
import transformers

In [ ]:
input_ids_comments = Input(shape=(128,), dtype=tf.int32, name='input_ids_comments')
input_mask_comments = Input(shape=(128,), dtype=tf.int32, name='attention_mask_comments')
input_ids_statements = Input(shape=(128,), dtype=tf.int32, name='input_ids_statements')
input_mask_statements = Input(shape=(128,), dtype=tf.int32, name='attention_mask_statements')

In [ ]:
bert_model = transformers.TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
bert_layer_comments = tf.keras.layers.Lambda(lambda x: bert_model(x[0], attention_mask=x[1])[0])([input_ids_comments, input_mask_comments])
bert_layer_statements = tf.keras.layers.Lambda(lambda x: bert_model(x[0], attention_mask=x[1])[0])([input_ids_statements, input_mask_statements])

In [ ]:
gru_output_comments = Bidirectional(GRU(64))(bert_layer_comments)
gru_output_statements = Bidirectional(GRU(64))(bert_layer_statements)

In [ ]:
concatenated_output = Concatenate()([gru_output_comments, gru_output_statements])

In [ ]:

output = Dense(6, activation='softmax')(concatenated_output)

In [ ]:
model = Model(inputs=[input_ids_comments, input_mask_comments, input_ids_statements, input_mask_statements], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
X_train_comments_ids = tf.squeeze(tf.stack([x['input_ids'] for x in X_train_comments]))
X_train_comments_masks = tf.squeeze(tf.stack([x['attention_mask'] for x in X_train_comments]))
X_train_statements_ids = tf.squeeze(tf.stack([x['input_ids'] for x in X_train_statements]))
X_train_statements_masks = tf.squeeze(tf.stack([x['attention_mask'] for x in X_train_statements]))

In [ ]:

X_test_comments_ids = tf.squeeze(tf.stack([x['input_ids'] for x in X_test_comments]))
X_test_comments_masks = tf.squeeze(tf.stack([x['attention_mask'] for x in X_test_comments]))
X_test_statements_ids = tf.squeeze(tf.stack([x['input_ids'] for x in X_test_statements]))
X_test_statements_masks = tf.squeeze(tf.stack([x['attention_mask'] for x in X_test_statements]))

In [ ]:

history = model.fit(
    [X_train_comments_ids, X_train_comments_masks, X_train_statements_ids, X_train_statements_masks],
    y_train,
    validation_data=([X_test_comments_ids, X_test_comments_masks, X_test_statements_ids, X_test_statements_masks], y_test),
    epochs=1,
    batch_size=32
)

In [ ]:
results = model.evaluate([X_test_comments_ids, X_test_comments_masks, X_test_statements_ids, X_test_statements_masks], y_test)
print("Test Loss, Test Accuracy:", results)

In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report
y_pred = model.predict([X_test_comments_ids, X_test_comments_masks, X_test_statements_ids, X_test_statements_masks])
y_pred_labels = y_pred.argmax(axis=1)
y_true_labels = y_test.values.argmax(axis=1)

print("Classification Report:")
print(classification_report(y_true_labels, y_pred_labels))
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_true_labels, y_pred_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")